In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
import collections
import witwidget

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import shuffle
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
# Download database
!gsutil cp 'gs://mortgage_dataset_files/mortgage-small.csv' .

Copying gs://mortgage_dataset_files/mortgage-small.csv...
| [1 files][330.8 MiB/330.8 MiB]                                                
Operation completed over 1 objects/330.8 MiB.                                    


In [3]:
# Create dictionary of database's column names
COLUMN_NAMES = collections.OrderedDict({
 'as_of_year': np.int16,
 'agency_code': 'category',
 'loan_type': 'category',
 'property_type': 'category',
 'loan_purpose': 'category',
 'occupancy': np.int8,
 'loan_amt_thousands': np.float64,
 'preapproval': 'category',
 'county_code': np.float64,
 'applicant_income_thousands': np.float64,
 'purchaser_type': 'category',
 'hoepa_status': 'category',
 'lien_status': 'category',
 'population': np.float64,
 'ffiec_median_fam_income': np.float64,
 'tract_to_msa_income_pct': np.float64,
 'num_owner_occupied_units': np.float64,
 'num_1_to_4_family_units': np.float64,
 'approved': np.int8
})

In [4]:
# Prepare data
data = pd.read_csv('mortgage-small.csv', index_col=False, dtype=COLUMN_NAMES)
data = data.dropna()
data = shuffle(data, random_state=2)

In [5]:
# View data
data.head()

,as_of_year,agency_code,loan_type,property_type,loan_purpose,occupancy,loan_amt_thousands,preapproval,county_code,applicant_income_thousands,purchaser_type,hoepa_status,lien_status,population,ffiec_median_fam_income,tract_to_msa_income_pct,num_owner_occupied_units,num_1_to_4_family_units,approved
310650,2016,Consumer Financial Protection Bureau (CFPB),"Conventional (any loan other than FHA, VA, FSA...",One to four-family (other than manufactured ho...,Refinancing,1,110.0,Not applicable,119.0,55.0,Freddie Mac (FHLMC),Not a HOEPA loan,Secured by a first lien,5930.0,64100.0,98.81,1305.0,1631.0,1
630129,2016,Department of Housing and Urban Development (HUD),"Conventional (any loan other than FHA, VA, FSA...",One to four-family (other than manufactured ho...,Home purchase,1,480.0,Not applicable,33.0,270.0,Loan was not originated or was not sold in cal...,Not a HOEPA loan,Secured by a first lien,4791.0,90300.0,144.06,1420.0,1450.0,0
715484,2016,Federal Deposit Insurance Corporation (FDIC),"Conventional (any loan other than FHA, VA, FSA...",One to four-family (other than manufactured ho...,Refinancing,2,240.0,Not applicable,59.0,96.0,"Commercial bank, savings bank or savings assoc...",Not a HOEPA loan,Secured by a first lien,3439.0,105700.0,104.62,853.0,1076.0,1
887708,2016,Office of the Comptroller of the Currency (OCC),"Conventional (any loan other than FHA, VA, FSA...",One to four-family (other than manufactured ho...,Refinancing,1,76.0,Not applicable,65.0,85.0,Loan was not originated or was not sold in cal...,Not a HOEPA loan,Secured by a subordinate lien,3952.0,61300.0,90.93,1272.0,1666.0,1
719598,2016,National Credit Union Administration (NCUA),"Conventional (any loan other than FHA, VA, FSA...",One to four-family (other than manufactured ho...,Refinancing,1,100.0,Not applicable,127.0,70.0,Loan was not originated or was not sold in cal...,Not a HOEPA loan,Secured by a first lien,2422.0,46400.0,88.37,650.0,1006.0,1


In [6]:
# View distribution of approved/denied mortgage applications
print(data['approved'].value_counts())

labels = data['approved'].values
data = data.drop(columns=['approved'])

1    665389
0    334610
Name: approved, dtype: int64


In [7]:
# Create dummy columns for categorical data
dummy_columns = list(data.dtypes[data.dtypes == 'category'].index)
data = pd.get_dummies(data, columns=dummy_columns)

In [8]:
# View modified data
data.head()

,as_of_year,occupancy,loan_amt_thousands,county_code,applicant_income_thousands,population,ffiec_median_fam_income,tract_to_msa_income_pct,num_owner_occupied_units,num_1_to_4_family_units,...,"purchaser_type_Life insurance company, credit union, mortgage bank, or finance company",purchaser_type_Loan was not originated or was not sold in calendar year covered by register,purchaser_type_Other type of purchaser,purchaser_type_Private securitization,hoepa_status_HOEPA loan,hoepa_status_Not a HOEPA loan,lien_status_Not applicable (purchased loans),lien_status_Not secured by a lien,lien_status_Secured by a first lien,lien_status_Secured by a subordinate lien
310650,2016,1,110.0,119.0,55.0,5930.0,64100.0,98.81,1305.0,1631.0,...,0,0,0,0,0,1,0,0,1,0
630129,2016,1,480.0,33.0,270.0,4791.0,90300.0,144.06,1420.0,1450.0,...,0,1,0,0,0,1,0,0,1,0
715484,2016,2,240.0,59.0,96.0,3439.0,105700.0,104.62,853.0,1076.0,...,0,0,0,0,0,1,0,0,1,0
887708,2016,1,76.0,65.0,85.0,3952.0,61300.0,90.93,1272.0,1666.0,...,0,1,0,0,0,1,0,0,0,1
719598,2016,1,100.0,127.0,70.0,2422.0,46400.0,88.37,650.0,1006.0,...,0,1,0,0,0,1,0,0,1,0


In [9]:
# Split data into training and testing sets
x, y = data.values, labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [10]:
# Define the model
model = xgb.XGBClassifier(
    objective='reg:logistic'
)

In [11]:
# Train the model
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='reg:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
# Test model on new data and determine accuracy
y_pred = model.predict(x_test)
acc = accuracy_score(y_test, y_pred.round())
print(acc, '\n')

0.872456 



In [13]:
model.save_model('model.bst')

In [16]:
# Set up test examples for What-If Tool
num_WIT_examples = 500
test_examples = np.hstack((x_test[:num_WIT_examples],y_test[:num_WIT_examples].reshape(-1,1)))

In [ ]:
# Initialize What-If Tool
config_builder = (WitConfigBuilder(test_examples.tolist(), data.columns.tolist() + ['mortgage_status'])
  .set_custom_predict_fn(model.predict_proba)
  .set_target_feature('mortgage_status')
  .set_label_vocab(['denied', 'approved']))
WitWidget(config_builder, height=800)